In [2]:
api_key = "ACCuu5P8p7oC3"

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


addresses = {
#             "kcs":"0xf34960d9d60be18cc1d5afc1a6f012a723a28811",
#             "id":"0xebd9d99a3982d547c5bb4db7e3b1f9f14b67eb83",
#             "zcn":"0xb9ef770b6a5e12e45983c5d80545258aa38f3b78",
#             "rfuel":"0xaf9f549774ecedbd0966c52f250acc548d3f36e5",
#             "celr":"0x4f9254c83eb525f9fcf346490bbb3ed28a81c667",
#             "rsr":"0x8762db106b2c2a0bccb3a80d1ed41273552616e8",
#             "orn":"0x0258f474786ddfd37abce6df6bbb1dd5dfc4434a",
# #             "ankr": "0x8290333cef9e6d528dd5618fb97a76f268f3edd4",
#             "ogn":"0x8207c1ffc5b6804f6024322ccf34f29c3541ae26",
#             "matic":"0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0",
#             "omg":"0xd26114cd6ee289accf82350c8d8487fedb8a0c07",
#             "uos": "0xd13c7342e1ef687c5ad21b27c2b65d772cab5c8c",
# #             "pond": "0x57b946008913b82e4df85f501cbaed910e58d26c",
#             "elf":"0xbf2179859fc6d5bee9bf9158632dc51678a4100e", 
#             "skl":"0x00c83aecc790e8a4453e5dd3b0b4b3680501a7a7",
#             "rep":"0x221657776846890989a759ba2973e427dff5c9bb"
#                "hot":"0x6c6ee5e31d828de241282b9606c8e98ea48526e2",
#                 "mtl":"0xF433089366899D83a9f26A773D59ec7eCF30355e",
#                 "inj":"0xe28b3b32b6c345a34ff64674606124dd5aceca30",
#                 "mana":"0x0f5d2fb29fb7d3cfee444a200298f468908cc942",
#                 "qnt":"0x4a220e6096b25eadb88358cb44068a3248254675",
#                 "dnt":"0x0abdace70d3790235af448c88547603b945604ea"
                "snx":'0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f',
            }
url = 'https://api.bloxy.info/token/token_holders_list'

responses = {}
for token, tkn_add in addresses.items():
    parameters = {
      'token':tkn_add,
      'key':api_key,
      'limit': 2000
    }
    headers = {
      'Accepts': 'application/json',
    }


    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        responses[token] = data
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

In [6]:
data

[{'address': '0xda4ef8520b1a57d7d63f1e249606d1a459698876',
  'address_type': 'Smart Contract',
  'annotation': '',
  'balance': 71904178.7229486},
 {'address': '0x8ca24021e3ee3b5c241bbfcee0712554d7dc38a1',
  'address_type': 'Smart Contract',
  'annotation': '',
  'balance': 11979088.880805893},
 {'address': '0x49be88f0fcc3a8393a59d3688480d7d253c37d2a',
  'address_type': 'Wallet',
  'annotation': 'Foundation, Synthetix',
  'balance': 11971129.40581554},
 {'address': '0x5fd79d46eba7f351fe49bff9e87cdea6c821ef9f',
  'address_type': 'Smart Contract',
  'annotation': '',
  'balance': 5533283.718417386},
 {'address': '0xe7178db9e123f053aabfd2c28520bf6b63cb4a47',
  'address_type': 'Smart Contract',
  'annotation': '',
  'balance': 5334132.180816593},
 {'address': '0x27cc4d6bc95b55a3a981bf1f1c7261cda7bb0931',
  'address_type': 'Wallet',
  'annotation': '',
  'balance': 4730054.156476632},
 {'address': '0x767ecb395def19ab8d1b2fcc89b3ddfbed28fd6b',
  'address_type': 'Wallet',
  'annotation': '',


In [4]:
from pathlib import Path
from datetime import date
file_name = 'bloxy/{}_1.json'.format(str(date.today()))

my_file = Path(file_name)
i = 2
while my_file.is_file():
    file_name = file_name.replace('.json','')[:-1] + str(i) + '.json'
    my_file = Path(file_name)
    i +=1
with open(my_file, 'w') as f:
    f.write(json.dumps(responses))

In [5]:
cov_id_map = {
"eth":1,
"matic":137,
"avalanche":43114,
"binance":56}


In [ ]:
holders = []
parameters = {
  'key':api_key
}
headers = {
  'Accepts': 'application/json',
}
session = Session()
session.headers.update(headers)

from tqdm import tqdm
for token, token_holders in responses.items():
    for holder_idx, holder_details in tqdm(enumerate(token_holders)):
        if holder_idx>=57:
            if holder_details['address_type'].lower() == 'wallet' and holder_details['annotation'] == '':
                holder_address = holder_details['address'] 

                try:
                    url = "https://api.covalenthq.com/v1/1/address/{}/portfolio_v2/".format(holder_address)
                    response = session.get(url, params=parameters)
                    data = json.loads(response.text)
                    holders.append({'token':token, 'holder_address': holder_address, 'data': data})
                except (ConnectionError, Timeout, TooManyRedirects, JSONDecodeError) as e:
                    print(e,token )

58it [00:02, 25.13it/s]

In [ ]:
from pathlib import Path

file_name = 'covalent/token_holders.json'

my_file = Path(file_name)
i = 2
while my_file.is_file():
    file_name = file_name.replace('.json','')[:-1] + str(i) + '.json'
    my_file = Path(file_name)
    i +=1
with open(my_file, 'w') as f:
    f.write(json.dumps(holders))


In [ ]:
from dateutil.parser import parse
import math
data_list = []
zero_rate = 0
for holder in holders:
    data = holder['data']
    root_token = holder['token']
    items = data['items']
    record_time = parse(data['updated_at']).date()
    holder_address = holder['holder_address']
    for contract in items: 
        
        token_name = contract['contract_name']
        token_symbol = contract['contract_ticker_symbol'].lower()
        if token_symbol == 'matic':
            continue
        holdings = contract['holdings']
        oldest_record = holdings[-1]
        newest_record = holdings[0]
        decimals = contract['contract_decimals']
        or_balance = int(oldest_record['close']['balance'])
        or_balance = math.pow(10, -decimals) * or_balance
        nw_balance = int(newest_record['close']['balance'])
        nw_balance = math.pow(10, -decimals) * nw_balance
        or_quote = float(oldest_record['close']['quote']) if oldest_record['close']['quote'] else 0
        nw_quote = float(newest_record['close']['quote']) if newest_record['close']['quote'] else 0
        or_rate = float(oldest_record['quote_rate'])  if oldest_record['quote_rate'] else 0
        nw_rate = float(newest_record['quote_rate']) if newest_record['quote_rate'] else 0
        or_date = parse(oldest_record['timestamp']).date()
        nw_date = parse(newest_record['timestamp']).date()
        
        if nw_rate ==0 and or_rate ==0:
            rate_change =0 
        else:
            rate_change = round(100 * (nw_rate - or_rate)/ (or_rate + 0.00000000000000000000000000000000000000000001), 1)
        
        rank = -1
        alive = 'NA'
        type_ = "NA"
        is_active = "NA"
        is_new = "NA"
        try:
            rank_data = idx_coin_data[token_symbol]
            
            rank = rank_data['rank']
            is_new = rank_data['is_new']
            type_ = rank_data['type']
            is_active = rank_data['is_active']
            
        except KeyError:
            pass
        
        
        ## cowboy coding but whatever
        positive_holdings = 0
        for contract in items:
            if int(contract['holdings'][0]['close']['balance'])>0:
                positive_holdings += 1
            
        output = { 
                    "holder_address": holder_address,
                    "root_token": root_token,
                    "record_time": record_time,
                    "token_name": token_name,
                    "token_symbol": token_symbol,
                    "or_balance": or_balance,
                    "nw_balance": nw_balance,
                    "or_quote": or_quote,
                    "nw_quote": nw_quote,
                    "or_rate": or_rate,
                    "nw_rate": nw_rate,
                    "or_date": or_date,
                    "nw_date": nw_date,
                    "rate_change": rate_change,
                    "num_holdings": positive_holdings, 
                    "rank": rank, 
                    'is_new': is_new,
                    'type_': type_,
                    'is_active': is_active,
                    'is_root': root_token == token_symbol
            
        }
        data_list.append(output)
        
holders_df = pd.DataFrame(data_list)

In [ ]:
print('pre_filtering', holders_df.shape)
filtered_df = holders_df.query('num_holdings  > 4 & num_holdings < 40 & is_root == False & is_active == True and rate_change > 50')
print('post_filtering', filtered_df.shape)
len(filtered_df.holder_address.unique())
watch_df = holders_df.query('or_quote  > 0 & is_root == False & nw_quote>10000')
watch_df = watch_df[watch_df['holder_address'].isin(filtered_df.holder_address.unique())]
multiple_wins = watch_df[watch_df['rate_change']>50].groupby('holder_address').count().sort_values(by=['rate_change'], ascending=False)
multiple_wins = multiple_wins[multiple_wins['rate_change'] > 1]


In [ ]:
multiple_wins_addresses = watch_df[watch_df['holder_address'].isin(multiple_wins.index)].groupby('holder_address').count()
pd.merge(multiple_wins_addresses['rate_change'], multiple_wins['rate_change'], on=multiple_wins.index)
# .to_csv("/home/hq16960/blg_projects/covid19/stock_cluster/matic_holders_v1.csv")